In [1]:
import hanlp
import random, re, traceback, joblib
import numpy as np
from sklearn.naive_bayes import MultinomialNB,GaussianNB,BernoulliNB


In [2]:
#加载停用词词典
def load_file(filepath):
    with open(filepath, 'r', encoding="utf-8") as f:
        contents = f.readlines()
    result = []
    for content in contents:
        result.append(content.strip())
    return result


def remove_stop_words(text, filepath):
    result = []
    str = ''
    dic = load_file(filepath)
    for k in text:
        for i in k:
            if i not in dic:
                result.append(i)
    # 对否定词“不、太”做特殊处理，把它与后面的词拼接
    while '不' in result:
        index = result.index('不')
        if index == len(result) - 1:
            break
        result[index:index + 2] = [''.join(result[index:index + 2])]
    str = " ".join(result)

    return str



def load_corpus(aaa):
    """
    df批处理
    """
    data = []
    for line in aaa:
        content = Hanlp(line)
        data.append(content)
    return data

In [3]:
# 调用hanlp进行分词
hanlp.pretrained.tok.ALL
tok = hanlp.load(hanlp.pretrained.tok.COARSE_ELECTRA_SMALL_ZH)

In [4]:
# 并行处理，将长评论进行分句同时处理
# 第一级管道分句，第二级管道分词和去停用词：
Hanlp = hanlp.pipeline().append(hanlp.utils.rules.split_sentence).append(tok).append(
    lambda sents: remove_stop_words(sents, '../../stop.txt'))
print(Hanlp(
    '元旦拜佛、天气好、气氛浓郁。但是导游的组织能力不够、拜佛时间20左右、出来等团队人员时间20分钟以上、有时等半小时都有。虽然根本原因是团队人员不遵守时间、但是作为组织人员应该具有对应方法。每次大多数人等那么几个不遵守时间的人而耽误时间。刚开始还想“大家出来玩不容易。理解理解”到每次等时间的延长、最后的忍无可忍了。一分钱一分货、便宜没好货、这句话是真理。'))

元旦 拜佛 天气 气氛 浓郁 导游 组织 能力 不够 拜佛 时间 20 左右 团队 人员 时间 20 分钟 半 小时 根本 原因 团队 人员 不遵守 时间 组织 人员 应该 具有 对应 方法 每次 大多数 几个 不遵守 时间 耽误 时间 刚 想 玩 不容易 理解 理解 每次 时间 延长 最后 忍无可忍 分 钱 分 货 便宜 没 货 句 话 真理


In [5]:
import pandas as pd

df = pd.read_csv('../../csv/moredata_label.csv')
df['content'] = load_corpus(df['content'])
df.head()


,content,score
0,海南 保亭 黎 苗族 风情 景区 看到 艺术 居住 环境 房屋 传统 表演 大型 艺术 表演...,0
1,槟榔 谷 知道 槟榔 知道 海南 吃 槟榔 习惯 初 三亚 听说 出租车 司机 窗外 吐 槟...,0
2,黎 苗族 文化 主题 公园 展示 黎族 苗族 服饰 手工 文化 传统 民居 狩猎 技能 传统...,0
3,那天 太阳 带 儿子 一起 里面 主要 少数民族 传统 习俗 文化 遗产 导游 讲解 一下 ...,0
4,景点 海南 游 中 比较 值得 地方 民族 歌舞 表演 值得 几家 朋友 驾 游 本来 想 ...,0


In [6]:
df_train = df.sample(frac=0.6, replace=False, random_state=0, axis=0)
df_test = df[~df.index.isin(df_train.index)]
print(df_train, df_test)

                                                content  score
3684                                           主要 参观 参观      0
5453                                      免费 海滩 环境 般 椰林      0
3140      三 江 入 海 玉带 滩 一边 江 平静 女孩 般 静谧 一边 大海 气势磅礴 男人 雄壮      0
144   呀诺达 槟榔 谷 真 觉得 更 像是 大型 旅游 商店 游览 车 每到 处 先 穿过 商店 ...      0
2201                                     不错 交通 不太 方便 专车      0
...                                                 ...    ...
5475     十一 期间 很少 挺 安静 个人 觉得 不错 水质 不干净 北戴河 差不多 沙子 算是 细腻      0
722   建议 景区 顾 导游 80 元 沿途 讲解 特别 每到 整 点 景区 工作人员 跳舞 挺 不...      0
4076                    总体来说 大坑 当日 购买 小时 生效 不太 不能 即时 生效      1
3156                                 论坛 会址 宏伟 壮观 环境美 心醉      0
2155                               没啥 意思 不值 专门 顺道 话 试 试      0

[3746 rows x 2 columns]                                                 content  score
0     海南 保亭 黎 苗族 风情 景区 看到 艺术 居住 环境 房屋 传统 表演 大型 艺术 表演...      0
6     今天 第一站 来到 计划 中 远 距离 槟榔 谷 说 实话 今天 天气 不力 18 176 ...      0
7     槟榔 谷 顾名思义 这是 槟榔 树 主题 观赏 

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(token_pattern='\[?\w+\]?')
#vectorizer = CountVectorizer(max_df=100)
x_train = vectorizer.fit_transform(df_train['content'])
y_train = df_train['score']
x_test = vectorizer.transform(df_test['content'])
y_test = df_test['score']

In [8]:
#读取测试集
tf = pd.read_csv('../../csv/test_label.csv')
tf['content'] = load_corpus(tf['content'])
tx_test= vectorizer.transform(tf['content'])
print(tf['content'])

0      " 整个 行程 安排 非常 合理 性 价 高 湘西 新 认识 张家界 凤凰 真的 值得 导游...
1      " 本来 打算 自助游 网上 热 评 果断 选择 团 证实 明智 举 周导 地地道道 土家 ...
2      " 行程 安排 完全 符合 购买 内容 说 实话 有点 累 理解 毕竟 天数 少 地方 景色...
3      " 行程 安排 合理 会 安排 到位 吃 住 玩 爽 真心 不错 没 玩 朋友 玩 朋友 相...
4      吕导 吕 泡菜 为人 随和 讲解 专业 角度 独到 服务 周到 特别 点 景点 故事 结合 ...
                             ...                        
290    时间 安排 成 问题 等待 坑 早上 五 点 起来 说 滑雪场 排队 导游 小时 导游 素质...
291    宜昌 市区 座 船 路 导游 说 两天 中餐 第一天 晚餐 不包含 团费 里面 外面 吃 快...
292    神农架 景色 不错 旅行社 服务 不怎么样 第一天 武汉 导游 订 九点 宜昌 票 宜昌 旅...
293    预定 两天 行程 兴山县 修路 不旅游 车 走 硬是 游船 困 3个小时 左右 导致 第一天...
294    風景 不錯 , 行程 安排 行 , 酒店 不太 乾淨 , 回程 火車票 安排 太 差 , 七...
Name: content, Length: 295, dtype: object


# 伯努利贝叶斯

In [9]:
# 训练
from sklearn.naive_bayes import BernoulliNB

clf = BernoulliNB()
# xxx=x_train.todense()
# yyy = y_train

# clf.fit(xxx, yyy)
clf.fit(x_train, y_train)
y_pre = clf.predict(x_test)
yy_pre = clf.predict_proba(x_test)
# 测试
from sklearn import metrics

print(metrics.classification_report(y_test, y_pre))
print('准确率:', metrics.accuracy_score(y_test, y_pre))

              precision    recall  f1-score   support

           0       0.81      0.96      0.88      1934
           1       0.61      0.21      0.32       564

    accuracy                           0.79      2498
   macro avg       0.71      0.59      0.60      2498
weighted avg       0.76      0.79      0.75      2498

准确率: 0.7914331465172137


In [10]:
tf.head()
ty_test = tf['score']
ty_pre = clf.predict(tx_test.todense())
print(metrics.classification_report(ty_test, ty_pre))
print('准确率:', metrics.accuracy_score(ty_test, ty_pre))

              precision    recall  f1-score   support

           0       0.80      0.91      0.85       174
           1       0.85      0.68      0.75       121

    accuracy                           0.82       295
   macro avg       0.82      0.80      0.80       295
weighted avg       0.82      0.82      0.81       295

准确率: 0.8169491525423729


C:\Users\Aris\anaconda3\envs\bishe\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


# 高斯贝叶斯结果

In [11]:
# 训练
from sklearn.naive_bayes import GaussianNB

clf = GaussianNB()
xxx=x_train.todense()
yyy = y_train
# xxx = pd.Series.to_numpy(x_train)
# print(xxx)
# yyy = pd.Series.to_numpy(y_train)
# print(yyy)
clf.fit(xxx, yyy)
# clf.fit(x_train, y_train)
y_pre = clf.predict(x_test.todense())
yy_pre = clf.predict_proba(x_test.todense())

# 测试
from sklearn import metrics

print(metrics.classification_report(y_test, y_pre))
print('准确率:', metrics.accuracy_score(y_test, y_pre))

C:\Users\Aris\anaconda3\envs\bishe\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\Aris\anaconda3\envs\bishe\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\Aris\anaconda3\envs\bishe\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


              precision    recall  f1-score   support

           0       0.82      0.66      0.73      1934
           1       0.31      0.52      0.39       564

    accuracy                           0.63      2498
   macro avg       0.57      0.59      0.56      2498
weighted avg       0.71      0.63      0.65      2498

准确率: 0.6269015212169736


In [12]:
tf.head()
ty_test = tf['score']
ty_pre = clf.predict(tx_test.todense())
print(metrics.classification_report(ty_test, ty_pre))
print('准确率:', metrics.accuracy_score(ty_test, ty_pre))

              precision    recall  f1-score   support

           0       0.61      0.76      0.68       174
           1       0.46      0.29      0.36       121

    accuracy                           0.57       295
   macro avg       0.53      0.53      0.52       295
weighted avg       0.55      0.57      0.54       295

准确率: 0.5694915254237288


C:\Users\Aris\anaconda3\envs\bishe\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


# 多项式贝叶斯结果

In [13]:
# 训练
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()
clf.fit(x_train, y_train)
y_pre = clf.predict(x_test)
yy_pre = clf.predict_proba(x_test)

# 测试
from sklearn import metrics

print(metrics.classification_report(y_test, y_pre))
print('准确率:', metrics.accuracy_score(y_test, y_pre))

              precision    recall  f1-score   support

           0       0.85      0.96      0.90      1934
           1       0.76      0.42      0.55       564

    accuracy                           0.84      2498
   macro avg       0.81      0.69      0.72      2498
weighted avg       0.83      0.84      0.82      2498

准确率: 0.8402722177742193


In [16]:
tf.head()
ty_test = tf['score']
ty_pre = clf.predict(tx_test)
print(metrics.classification_report(ty_test, ty_pre))
print('准确率:', metrics.accuracy_score(ty_test, ty_pre))

              precision    recall  f1-score   support

           0       0.88      0.96      0.92       174
           1       0.93      0.82      0.87       121

    accuracy                           0.90       295
   macro avg       0.91      0.89      0.90       295
weighted avg       0.90      0.90      0.90       295

准确率: 0.9016949152542373


In [17]:
joblib.dump(clf, 'clf_model6000.m')
joblib.dump(vectorizer,'vec_model.m')
print('ok!')

ok!


## 加载模型测试


In [ ]:
t_clf = joblib.load('clf_model6000.m')
t_df = pd.read_csv('../../csv/test_label.csv')
t_df['content'] = load_corpus(t_df['content'])
t_pre=t_clf.predict(vectorizer.transform(t_df['content']))
print(metrics.classification_report(t_df['score'], t_pre))
print('准确率:', metrics.accuracy_score(t_df['score'], t_pre))

In [ ]:
def load_text(aaa):
    """
    df批处理
    """
    # 并行处理，将长评论进行分句同时处理
    # 第一级管道分句，第二级管道分词和去停用词：
    data = []
    content = Hanlp(aaa)
    data.append(content)
    return data

In [ ]:
csa = clf.predict(vectorizer.transform(load_text('2天的行程基本上来说还是有点紧张，神农架的空气很清新，景色真的很美。')))
print(csa)

In [1]:
import joblib
clff = joblib.load('clf_model6000.m')
vecc = joblib.load('vec_model.m')
csa = clff.predict(vecc.transform(load_text("我们是提前预定的，1月9号就预定了，如果我早知道1月20号疫情会如此严重我绝对，绝对不会去预定这个行程。后来20号疫情严重化，本人又感冒，于是申请了退款，抱歉我们也是十分的不想退，携程客服打电话就一句话：退了，钱不退。我？？？于是我们迎难而上。***的操作来了，也就是景区，年初一，25号，因为***被迫关了，暂停接待，之前很多人都在建议关了，景区老总陈向宏发了微博，如图，别说我都感动的哭了，当然如果没有年初一的骚操作的话，我估计都会觉得大家都不容易。年初一，我们年三十提前一天的游客，就如同丧家犬一样，中午没有卖吃的了！！！对，景区卖吃的都关了，小景点也都关了。我们年初一就吃了一顿早饭，酒店自助。饿着肚子走的。如果我对退款不满意的是携程，是旅行社，那么景区这一波骚操作是人能干的事吗？游客还没走完，就关店了？想赚钱的时候，情怀就来了，说着如图所示的话，您03年记得别人无助的身影，为什么20年了，最后一天都不愿意服务前一天的游客呢？我们年三十入住的游客，迎难而上的游客，硬着头皮去的游客，已经抱着很好的心态去了，结果呢？是的，25号关就关，管24号游客是否能吃上饭，是否会有温暖。")))
print(csa)

NameError: name 'load_text' is not defined